In [1]:
%%HTML
<link rel="stylesheet" href="rise.css">
<link rel="stylesheet" href="custom.css">

# <span style="color:maroon"> Data assimilation, machine learning and dynamical systems - Part I  </span>
#### Marc Bocquet¹ [marc.bocquet@enpc.fr](mailto:marc.bocquet@enpc.fr) and Julien Brajard² [julien.brajard@nersc.no](mailto:julien.brajard@nersc.no)
#### (1) CEREA, École des Ponts et EdF R&D, Île-de-France, France
#### (2) Nansen Center (NERSC), Bergen, Norway

During this session, we will discover some connections between data assimilation and deep learning when applied to dynamical systems.

#  <span style="color:teal"> Objective of this MOOC and outline </span>

The goal of this lecture is to give a brief and very limited introduction to the connections between __machine learning/deep learning__ and __data assimilation__. Machine learning has found many new convincing applications over the past couple of years, besides computer vision or natural language. The geosciences are among them. Even within the geosciences, there is a considerable range of potential applications of machine learning and deep learning; some of them have been evidenced recently.

Our specific goal today will be to not only learn the state of a physical system through its observation and a prior of this state but also to correct its dynamics.This contrasts with traditional data assimilation where the model is usually assumed to be known, or corrected via only a bunch of parameters in the control variables. 

<span style="color:teal">__This MOOC is organised into four parts:__</span>

1. This theoretical introduction (intro), which you might be tempted to skip if you are not interested in the mathematical rationale.
2. The toy model that we will try to learn is introduced, assuming we already have an approximation. The *ensemble Kalman filter* data assimilation method is applied to the observations of the true model using our best approximation of this true model (part 1) and provides an analysis of the full true model trajectory.
3. A correction to the approximate model is then learned through *machine learning* using the approximate model and the analysis obtained from part 1 (part 2).
4. Finally, the *hybrid surrogate* model (i.e., the approximate model together with our machine learning correction) is tested and evaluated (part 3).

This will be followed by a small set of questions about the MOOC.

### <span style="color:teal"> For more information about the ideas developed and exemplified in this mooc, please have a look at the following papers: </span>

* <font size="3"> Bocquet, M., Brajard, J., Carrassi, A., & Bertino, L. (2019). Data
assimilation as a learning tool to infer ordinary differential equation
representations of dynamical models. *Nonlin. Processes Geophys.*, *26*,
143--162. https://doi.org/10.5194/npg-26-143-2019  </font> <br /> 

* <font size="3"> Bocquet, M., Brajard, J., Carrassi, A., & Bertino, L. (2020). Bayesian
inference of chaotic dynamics by merging data assimilation, machine
learning and expectation-maximization. *Foundations of Data Science*, *2*, 55--80. https://doi.org/10.3934/fods.2020004  </font> <br /> 

* <font size="3"> Brajard, J., Carrassi, A., Bocquet, M., & Bertino, L. (2020). Combining
data assimilation and machine learning to emulate a dynamical model from
sparse and noisy observations: a case study with the Lorenz 96 model. *J. Comput. Sci.*, *44*, 101171.
https://doi.org/10.1016/j.jocs.2020.101171  </font> <br /> 

* <font size="3"> Brajard, J., Carrassi, A., Bocquet, M., & Bertino, L. (2021). Combining
data assimilation and machine learning to infer unresolved scale
parametrisation. *Phil. Trans. R. Soc. A*, *379*, 20200086.
https://doi.org/10.1098/rsta.2020.0086  </font> <br /> 

* <font size="3"> Farchi, A., Bocquet, M., Laloyaux, P., Bonavita, M., & Malartic, Q.
(2021). A comparison of combined data assimilation and machine learning
methods for offline and online model error correction. *J. Comput.
Sci.*, *55*, 101468. https://doi.org/10.1016/j.jocs.2021.101468  </font> <br /> 

* <font size="3"> Farchi, A., Laloyaux, P., Bonavita, M., & Bocquet, M. (2021). Using
machine learning to correct model error in data assimilation and
forecast applications. *Q. J. R. Meteorol. Soc.*, *147*, 3067--3084.
https://doi.org/10.1002/qj.4116  </font>

# <span style="color:teal"> Preambule: theoretical fundamentals and objective </span>

## <span style="color:blue"> Dynamical system and observation </span>

Let us consider a dynamical system which is observed:
\begin{align*}
\mathbf{x}_{k} &= M_k(\mathbf{x}_{k-1}) + \eta_k, \\
\mathbf{y}_{k} &= H_k(\mathbf{x}_{k-1}) + \varepsilon_k,
\end{align*}
where $\mathbf{x}_k$ is the state vector at time $t_k$, $\mathbf{y}_k$ is the observation vector at time $t_k$,
$M_k$ is the numerical evolution model from time $t_{k-1}$ to time $t_k$, and $H_k$ is the observation operator. The noise vector $\varepsilon_k$ follows a Gaussian distribution and is meant to account for some of model error:
$\varepsilon_k \sim {\mathcal N}({\mathbf 0}, {\mathbf Q}_k)$. The noise vector $\eta_k$ follows a Gaussian distribution and is meant to account for observation error: $\eta_k \sim {\mathcal N}({\mathbf 0}, {\mathbf R}_k)$.

## <span style="color:blue"> Weak-constraint 4D-Var and generalisation </span>

Then the typical weak-constraint 4D-Var associated to this system is described by the cost function:
$$
J(\mathbf{x}_{0:K}) = - J_\mathrm{b}(\mathbf{x}_0) + \sum_{k=0}^K \|\mathbf{y}_k-H_k(\mathbf{x}_k) \|^2_{\mathbf{R}_k^{-1}}+ \sum_{k=1}^K \|\mathbf{x}_k- M_k\left(\mathbf{x}_{k-1}\right) \|^2_{\mathbf{Q}_k^{-1}}, 
$$
where $J_\mathrm{b}$ is the prior/background term, the second term is departure from the observations, and the third term is the model error term which measures the departure from what the model $M_k$ would predict.
Now, we can widen the problem and assume the model is at least partly statistical on depend a vector of constant-in-time parameters 
$\boldsymbol{\omega}$ to be adjusted if not learned, i.e.:
$$
\mathbf{x}_{k} = M_k(\boldsymbol{\omega}, \mathbf{x}_{k-1}) + \eta_k.
$$
They could be physical parameters of the model, or weights and biases of a neural network correction to a physical model. The generalised cost function becomes:
$$
J(\boldsymbol{\omega}, \mathbf{x}_{0:K}) = - J_\mathrm{b}(\mathbf{x}_0) + \sum_{k=0}^K \|\mathbf{y}_k-H_k (\mathbf{x}_k) \|^2_{\mathbf{R}_k^{-1}}+ \sum_{k=1}^K \|\mathbf{x}_k- M_k\left(\boldsymbol{\omega}, \mathbf{x}_{k-1}\right) \|^2_{\mathbf{Q}_k^{-1}}.
$$

## <span style="color:blue"> Combined data assimilation and machine learning </span>

### First step: data assimilation

Hence, we want to minimise this cost function both on the trajectory $\mathbf{x}_{0:K}$ and on the model statistical parameters. But they are very different in nature and quite difficult to minimise altogether. Hence we will split the task on first a data assimilation part focusing on $\mathbf{x}_k$, and then a machine learning task focusing on $\boldsymbol{\omega}$.
We first assume the model is of the form:
$$
M_k(\boldsymbol{\omega},\mathbf{x}_{k-1}) = \Phi(\mathbf{x}_{k-1}) + \mathrm{nn}(\boldsymbol{\omega}, \mathbf{x}_{k-1})
$$
where $\Phi$ is a known physical model, approximation of the true model and $\mathrm{nn}(\boldsymbol{\omega})$ 
is a neural network correction.
$$
J(\boldsymbol{\omega}, {\color{red}{\mathbf{x}_{0:K}}}) = - J_\mathrm{b}({\color{red}{\mathbf{x}_0}})+\sum_{k=0}^K \|\mathbf{y}_k-H_k ({\color{red}{\mathbf{x}_k}}) \|^2_{\mathbf{R}_k^{-1}}+\sum_{k=1}^K \| {\color{red}{\mathbf{x}_k}}- M_k\left(\boldsymbol{\omega}, {\color{red}{\mathbf{x}_{k-1}}}\right) \|^2_{\mathbf{Q}_k^{-1}}.
$$
At fixed $\boldsymbol{\omega}$, this is a smoothing data assimilation problem, which can be solved more or less acurately by any proper data assimilation method: 4D-Var, WC 4D-Var, EnKF, EnKS, IEnKS, etc.

## <span style="color:blue"> Combined data assimilation and machine learning </span>

### Second step: machine learning

Next, once a state trajectory $\mathbf{x}^\star_{0:K}$ has been obtained from data assimilation, one can focus on the machine learning problem and specifically $\boldsymbol{\omega}$:
$$
J({\color{red}{\boldsymbol{\omega}}}, \mathbf{x}^\star_{0:K}) = - J_\mathrm{b}(\mathbf{x}^\star_0) + \sum_{k=0}^K \|\mathbf{y}_k-H_k(\mathbf{x}^\star_k) \|^2_{\mathbf{R}_k^{-1}}+ \sum_{k=1}^K \| \mathbf{x}^\star_k - \Phi(\mathbf{x}^\star_{k-1}) - \mathrm{nn}({\color{red}{\boldsymbol{\omega}}},\mathbf{x}^\star_{k-1}) \|^2_{\mathbf{Q}_k^{-1}}.
$$
The output is a learned neural net $\mathrm{nn}(\boldsymbol{\omega^\star})$, and hence a correction of the hybrid physical/statistical model.

## <span style="color:blue"> Combined data assimilation and machine learning </span>

### Coordinate descent

Moreover, it can be shown that one can iterate those two steps, which is known as a coordinate descent, following the schematic:

![schematic](./figures/loop_daml.png "Fig.1 - Coordinate descent for DAML.")

Fig.1 - Coordinate descent schematic for DAML.
